
#Some Extra Information:

##Game Search
-Enter a search term from any of Title, Platform and Genre

##Cart
-When choosing games to rent, if valid, they are added to your cart before being rented, you will need to check out your cart before the rental actually takes place in the files. I quite liked this feature as it gives you the chance to look over the request before it is submitted.


In [1]:
import ipywidgets as widgets
from IPython.display import clear_output
import gameSearch as GS
import gameRent as GR
import gameReturn as GRN
import InventoryPruning as IP
import matplotlib.pyplot as plt
from datetime import datetime as DT
import datetime

In [4]:
cart = []
numberOfItems = 0

#//////////Game Search///////////#

def gameSearchClicked(b):
    inputForm = widgets.VBox([gameSearchLabel, widgets.HBox([searchBox,
                                                    searchsubmitBTN])])
    with output:
        clear_output()
        display(inputForm)

def searchsubmitBTNClicked(b):
    results = GS.searchForGame(searchBox.value)
    with output:
        clear_output()
        totalResults = widgets.Label("Total: " + str(len(results)))
        display(totalResults)
        for result in results:
            resultTable = widgets.HBox([widgets.Label(result[3]), 
                        widgets.Label("   -   Game ID: "+result[0])])
            display(resultTable)

#////////////Game Rent//////////#

def rentGameBTNClicked(b):
    searchRentBoxLabel = widgets.Label("Please enter Game ID")
    rentGameInput = widgets.VBox([searchRentBoxLabel,widgets.HBox([searchBoxRent, rentSubmitBTN])])
    with output:
        clear_output()
        display(rentGameInput)

def rentSearchClicked(b):
    global gameID
    gameID = searchBoxRent.value
    
    try:
        gameAvailable = GR.availableToRentGame(gameID)
        gameFoundLabel = widgets.Label("")
        with output:
            if(gameAvailable):
                clear_output()
                gameFoundLabel.value = "Game Available, Enter Customer ID..."
                display(widgets.VBox([gameFoundLabel, widgets.HBox([custIDBox,
                                                         custIDSubmitBTN])]))
            else:
                clear_output()
                gameFoundLabel.value = "Sorry! Game is not available right now"
                display(gameFoundLabel)
    except:
        with output:
            display(widgets.Label("Not a valid GameID! Try again..."))
    
def custIDSubmitBTNClicked(b):
    global custID
    custID = custIDBox.value
    with output:
        try:
            if(GR.verifyCustomer(custID)):
                cart.append(GS.searchForGame(gameID))
                clear_output()
                display(widgets.Label("Added to cart!"))
        except:
            display(widgets.Label("Invalid customer ID"))

#////////////Cart Button//////////#

def cartViewBTNClicked(b):
    with output:
        clear_output()
        if(len(cart) > 0):
            # listHeader = widgets.Label("ID - Name - Platform")
            display(widgets.VBox([rentBTN, clearBTN]))
            count = 0
            for game in cart:
                count+=1
                display(widgets.Label(str(count) + ": " +game[0][3]+ " | Game ID: " + game[0][0]))
        else:
            display(widgets.Label("No games added yet !"))

def clearBTNClicked(b):
    with output:
        cart.clear()
        cartViewBTNClicked(b)

def rentCartGames(b):
    with output:
        clear_output()
        print(cart)
        for game in cart:
            GR.gameBeingRented(game[0][0], custID)
        cart.clear()

#////////////Game Return//////////#

def gameReturnBTNClicked(b):
    enterCustIDLBL = widgets.Label("Enter Customer ID:")
    returnGameBox = widgets.VBox([enterCustIDLBL,widgets.HBox([custIDBoxReturn,
                                                            submitReturnBTN])])
    with output:
        clear_output()
        display(returnGameBox)

def submitReturnBTNClicked(b):
    global ReturnCustID
    ReturnCustID = custIDBoxReturn.value
    try:        
        isValid = GRN.custIDCheck(ReturnCustID)
        with output:
            clear_output()
            if(isValid):
                enterGameIDLBL = widgets.Label("Enter Game ID:")
                form = widgets.VBox([enterGameIDLBL,widgets.HBox(
                                    [gameIDReturnBox, gameIDReturnBTN])])
                display(form)
    except:
        with output:
            clear_output()
            display(widgets.Label("Not a valid customer ID"))
    

def gameIDReturnBTNClicked(b):
    global ReturnGameID
    ReturnGameID = gameIDReturnBox.value
    
    try:
        gameInList = GRN.verifyGameInList(ReturnCustID, ReturnGameID, 
                                GRN.returnListFromCustID(ReturnCustID))
        with output:
            clear_output()
            if(gameInList):
                gameName = GS.searchForGame(ReturnGameID)[0][3]
                msgstr = "Please provide feedback for the game "+ gameName+ ":"
                message = widgets.Label(msgstr)
                form = widgets.VBox([message,widgets.HBox([gameFeedbackBox, 
                                            ratingOption]),saveReviewBTN])
                display(form)
            else:
                display(widgets.Label("Customer does not have game"))
    except:
        with output:
            display(widgets.Label("Invalid game ID"))
    
    
def saveReviewBTNClicked(b):
    try:
        if(not (int(ratingOption.value) > 0 and int(ratingOption.value) < 6)):
            with output:
                display(widgets.Label("Not a valid rating! Try again..."))
        else:
            GRN.getFeedback(ReturnCustID, ReturnGameID, ratingOption.value, 
                            gameFeedbackBox.value)
            GRN.returnGame(ReturnCustID, ReturnGameID)
            with output:
                clear_output()
                display(widgets.Label("Game returned!"))
    except:
        with output:
            display(widgets.Label("Not a valid rating! Try again..."))

#///////////Inventory Button///////////#

def lowestPerformingClicked(b):

    data1 = IP.getLowest5Games(IP.getRentalHistory())
    data2 = IP.getFeedbackHistory()

    id1 = []
    dates = []
    id2 = []
    rating = []

    for record in data1:
            id1.append(record[0])
            date = DT.strptime(record[1], "%Y-%m-%d")
            dates.append(date)

    for record in data2:
            id2.append(record[0])
            rating.append(record[1])

    fig, grph = plt.subplots(nrows=2, ncols=1)
    fig.suptitle('Lowest Performing Games')

    grph[0].yaxis_date()
    grph[0].set_ylim(datetime.datetime(2019, 1, 1, 0, 0), 
                     datetime.datetime.today())
    grph[1].set_ylim(0.5,5.5)

    grph[0].set_xlabel('Game ID')
    grph[0].set_ylabel('Date')

    grph[1].set_xlabel('Game ID')
    grph[1].set_ylabel('Rating')

    grph[0].set_title('Last time game was rented')
    grph[1].set_title('Lowest ratings')
    
    grph[0].bar(id1, dates)
    grph[1].bar(id2, rating)

    fig.tight_layout()

    with output:
        clear_output()
        plt.show()

#////// Widgets //////#

searchForGameBTN = widgets.Button(description="Search for Game")
gameSearchLabel = widgets.Label("Enter Search Term:")
searchBox = widgets.Text()
searchsubmitBTN = widgets.Button(description="Search")

cartViewBTN = widgets.Button(description="View Cart")
rentBTN = widgets.Button(description="Rent Games")
clearBTN = widgets.Button(description="Clear Cart")

rentGameBTN = widgets.Button(description="Rent a Game")
searchBoxRent = widgets.Text()
rentSubmitBTN = widgets.Button(description="Search")
custIDBox = widgets.Text()
custIDSubmitBTN = widgets.Button(description="Search")

returnGameBTN = widgets.Button(description="Return a Game")
custIDBoxReturn = widgets.Text()
submitReturnBTN = widgets.Button(description="Search")
gameIDReturnBTN = widgets.Button(description="Submit")
gameIDReturnBox = widgets.Text()
gameFeedbackBox = widgets.Textarea()        
comboLayout = widgets.Layout(width="70px")
ratingOption = widgets.Combobox(options=['1','2','3','4','5'], 
                                layout=comboLayout)
saveReviewBTN = widgets.Button(description="Return Game")

SalesInfoBTN = widgets.Button(description="Sales Info")
ratingAverageDisplay = widgets.Label()

#/////// On click ////////#

SalesInfoBTN.on_click(lowestPerformingClicked)

custIDSubmitBTN.on_click(custIDSubmitBTNClicked)
rentGameBTN.on_click(rentGameBTNClicked)
rentSubmitBTN.on_click(rentSearchClicked)

searchForGameBTN.on_click(gameSearchClicked)
searchsubmitBTN.on_click(searchsubmitBTNClicked)

cartViewBTN.on_click(cartViewBTNClicked)
rentBTN.on_click(rentCartGames)
clearBTN.on_click(clearBTNClicked)

returnGameBTN.on_click(gameReturnBTNClicked)
submitReturnBTN.on_click(submitReturnBTNClicked)
gameIDReturnBTN.on_click(gameIDReturnBTNClicked)
saveReviewBTN.on_click(saveReviewBTNClicked)


buttons = widgets.HBox([searchForGameBTN, rentGameBTN, returnGameBTN, cartViewBTN, SalesInfoBTN])
output = widgets.Output()

display(buttons, output)

Output()